# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [52]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome("chromedriver.exe")

# Passo 1: Pegar a cotação do Dólar
navegador.get("https://www.google.com")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.google.com")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

5.5186
6.3768802650000005
311.45


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [53]:
# Passo 4: Importar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [54]:
# Passo 5: Atualizar a cotação, o Preço de Compra e o Preço de Venda

# - atualizar cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# - atualizar preço de compra = Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Base Original"] * tabela["Cotação"]

# - atualizar o preço de venda = Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final,Preço de Compra,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.51860,4999.95,1.40,6999.930,5518.544814,7725.962740
1,Carro Renault,4500.00,Euro,6.37688,27000.00,2.00,54000.000,28695.961193,57391.922385
2,Notebook Dell,899.99,Dólar,5.51860,4499.95,1.70,7649.915,4966.684814,8443.364184
3,IPhone,799.00,Dólar,5.51860,3995.00,1.70,6791.500,4409.361400,7495.914380
4,Carro Fiat,3000.00,Euro,6.37688,18000.00,1.90,34200.000,19130.640795,36348.217511
5,Celular Xiaomi,480.48,Dólar,5.51860,2402.40,2.00,4804.800,2651.576928,5303.153856
6,Joia 20g,20.00,Ouro,311.45000,7000.00,1.15,8050.000,6229.000000,7163.350000


### Agora vamos exportar a nova base de preços atualizada

In [55]:
# Passo 6: Exportar o relatório atualizado
tabela.to_excel("Produtos Novo.xlsx", index=False)
navegador.quit()